In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf


df = pd.read_csv('csv/output2.csv')


df.fillna(0, inplace=True)


label_encoder = LabelEncoder()
df['class_name'] = label_encoder.fit_transform(df['class_name'])


feature_columns = ['right_shoulder_angle', 'right_elbow_angle', 'right_hip_angle', 'right_knee_angle',
                   'left_shoulder_angle', 'left_elbow_angle', 'left_hip_angle', 'left_knee_angle']
target_column = 'class_name'


scaler = StandardScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])


X = df[feature_columns].values
y = df[target_column].values


timesteps = 40
num_samples = X.shape[0] // timesteps
X = X[:num_samples * timesteps].reshape(num_samples, timesteps, len(feature_columns))
y = y[:num_samples * timesteps].reshape(num_samples, timesteps)


y = y[:, -1]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data preparation completed.")


Data preparation completed.


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

def build_gru_model(input_shape, num_classes):
    model = Sequential()
    model.add(GRU(64, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(64))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    return model

input_shape = (timesteps, len(feature_columns))
num_classes = len(label_encoder.classes_)

gru_model = build_gru_model(input_shape, num_classes)
gru_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
gru_model.summary()


gru_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)
ss

C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 40, 64)         │        14,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 40, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,493 (154.27 KB)

 Trainable params: 39,493 (154.27 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 159ms/step - accuracy: 0.1970 - loss: 1.6151 - val_accuracy: 0.5652 - val_loss: 1.3898
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6409 - loss: 1.3390 - val_accuracy: 0.7826 - val_loss: 1.1931
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6892 - loss: 1.1877 - val_accuracy: 0.8696 - val_loss: 1.0269
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7266 - loss: 1.0131 - val_accuracy: 0.8261 - val_loss: 0.8825
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.7684 - loss: 0.8691 - val_accuracy: 0.8696 - val_loss: 0.7630
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8548 - loss: 0.7189 - val_accuracy: 0.8696 - val_loss: 0.6661
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7867 - loss: 0.7029 - val_accuracy: 0.8261 - val_loss: 0.5856
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8135 - loss: 0.5732 - val_accuracy: 0.8696 - val_loss: 0.5296

In [3]:

gru_loss, gru_accuracy = gru_model.evaluate(X_test, y_test)
print(f'GRU Model - Loss: {gru_loss}, Accuracy: {gru_accuracy}')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9310 - loss: 0.2895
GRU Model - Loss: 0.28948041796684265, Accuracy: 0.931034505367279


In [5]:

gru_model.save('gru_model.h5')
import joblib


joblib.dump(label_encoder, 'gru_label_encoder.pkl')
print("Classes used during training:", label_encoder.classes_)

Classes used during training: ['bicep' 'incline' 'lat' 'row' 'squat']
